In [1]:
from simplex import *
import sympy as sp
import numpy as np

## Linear and Integer Optimization: Chapter 5 Sensibility Analysis

In [125]:
def pivoting_symbolic(tableau: np.ndarray, row: int, col: int) -> np.ndarray:
    # escale pivot row min to 1.0
    tableau[row, :] = tableau[row, :] / tableau[row, col]
    # pivot proccess: convert al column to zero except row
    for k in range(tableau.shape[0]):
        if k != row:
            tableau[k, :] = tableau[k, :] - tableau[k, col] * tableau[row, :]

In [126]:
delta = sp.symbols("\\delta")
# update cost of the optimal tableau with perturbations
tableau = sp.Matrix([[3 + delta, 2, 0, 0, 0, 0, 0],
                     [0., 1., 1.5, -0.5, 0., 0., 4.5],
                     [1., 0., -0.5, 0.5, 0., 0., 4.5],
                     [0., 0., 0.5, -0.5, 1., 0., 2.5],
                     [0., 0., -1.5, 0.5, 0., 1., 1.5]])
# solution
basic_var = [1, 0, 4, 5]
# correct cost
pivoting_symbolic(tableau, 2, 0)
pivoting_symbolic(tableau, 1, 1)
sp.nsimplify(tableau, rational=True)

Matrix([
[0, 0, \delta/2 - 3/2, -\delta/2 - 1/2, 0, 0, -9*\delta/2 - 45/2],
[0, 1,            3/2,            -1/2, 0, 0,                9/2],
[1, 0,           -1/2,             1/2, 0, 0,                9/2],
[0, 0,            1/2,            -1/2, 1, 0,                5/2],
[0, 0,           -3/2,             1/2, 0, 1,                3/2]])

## Práctica Dirigida 6


#### 2. Sea la función $f : \mathbb{R} \rightarrow \mathbb{R}$ definida como

\begin{align*}
f(\delta) :=\max (1 + \delta)x_1& +& x_2&\\
s.t.\quad 2x_1& +& x_2& \le 8\\
3x_1& +& x_2& \le 10\\
{}& {}& x_2& \le 5
\end{align*}

$$x1 \ge 0, x2 \ge 0.$$

a) Determinar f en un intervalo conteniendo 0, apartir de la información de la tabla simplex optimal.

In [127]:
# Hallando la tabla óptima
A = np.array([[2, 1., 1, 0, 0],
              [3, 1., 0, 1, 0],
              [0, 1., 0, 0, 1]])

b = [8, 10., 5]

c = np.array([1, 1.0, 0, 0, 0])

basic_var = [2, 3, 4]

tableau = build_tableau(A, b, c)

simplex(tableau, basic_var);


vertex = [0. 0.], x_B = [3, 4, 5]
[[ 1.  1.  0.  0.  0.  0.]
 [ 2.  1.  1.  0.  0.  8.]
 [ 3.  1.  0.  1.  0. 10.]
 [ 0.  1.  0.  0.  1.  5.]]
vertex = [3.33333333 0.        ], x_B = [3, 1, 5]
[[ 0.          0.66666667  0.         -0.33333333  0.         -3.33333333]
 [ 0.          0.33333333  1.         -0.66666667  0.          1.33333333]
 [ 1.          0.33333333  0.          0.33333333  0.          3.33333333]
 [ 0.          1.          0.          0.          1.          5.        ]]
vertex = [2. 4.], x_B = [2, 1, 5]
[[ 0.  0. -2.  1.  0. -6.]
 [ 0.  1.  3. -2.  0.  4.]
 [ 1.  0. -1.  1.  0.  2.]
 [ 0.  0. -3.  2.  1.  1.]]
vertex = [1.5 5. ], x_B = [2, 1, 4]
[[ 0.   0.  -0.5  0.  -0.5 -6.5]
 [ 0.   1.   0.   0.   1.   5. ]
 [ 1.   0.   0.5  0.  -0.5  1.5]
 [ 0.   0.  -1.5  1.   0.5  0.5]]


b) Determinar la tabla simplex optimal en el intervalo determinado en a)


In [128]:
# PRIMERA FORMA: tabla optima con los nuevos pesos
tableau = sp.Matrix([[1 + delta, 1, 0, 0., 0, 0],
                    [0., 1., 0., 0., 1., 5.],
                    [1., 0., 0.5, 0., -0.5, 1.5],
                    [0., 0., -1.5, 1., 0.5, 0.5]])
basic_var = [1, 0, 3]

pivoting_symbolic(tableau, 2, 0)
pivoting_symbolic(tableau, 1, 1)

sp.nsimplify(tableau, rational=True)

Matrix([
[0, 0, -\delta/2 - 1/2, 0, \delta/2 - 1/2, -3*\delta/2 - 13/2],
[0, 1,               0, 0,              1,                  5],
[1, 0,             1/2, 0,           -1/2,                3/2],
[0, 0,            -3/2, 1,            1/2,                1/2]])

In [129]:
# SEGUNDA FORMA
non_basic_var = [2, 4]
# B_inv = np.linalg.inv(A[:, basic_var])
# N = A[:, non_basic_var]
B_invxN = tableau[1:, non_basic_var]
# costos perturbados
c = sp.Matrix([1 + delta, 1, 0, 0., 0])
c_BI = c.row(basic_var)
c_NI = c.row(non_basic_var)

range_delta = sp.nsimplify(c_NI.T - c_BI.T @ B_invxN)
range_delta

Matrix([[-\delta/2 - 1/2, \delta/2 - 1/2]])

c) Gráficamente hallar los vértices de la restricción, para luego hallar explicitamente el valor de f

In [130]:
V = sp.Matrix([[0, 0], [0, 5], [3 / 2, 5], [2, 4], [10 / 3, 0]])
cost = sp.Matrix([1 + delta, 1])
sp.nsimplify(V@cost)


Matrix([
[                 0],
[                 5],
[ 3*\delta/2 + 13/2],
[      2*\delta + 6],
[10*\delta/3 + 10/3]])

## Práctica Calificada 5


In [131]:
A = [[1, 1, 1., 1, 0, 0],
     [2, 1, 3., 0, 1, 0],
     [1, 2, -1, 0, 0, 1]]

b = [2, 4, 2.]

c = [1, 2, -2, 0, 0, 0]

basic_var = [3, 4, 5]
tableau = build_tableau(A, b, c)

simplex(tableau, basic_var);


vertex = [0. 0. 0.], x_B = [4, 5, 6]
[[ 1.  2. -2.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.  0.  2.]
 [ 2.  1.  3.  0.  1.  0.  4.]
 [ 1.  2. -1.  0.  0.  1.  2.]]
vertex = [0. 1. 0.], x_B = [4, 5, 2]
[[ 0.   0.  -1.   0.   0.  -1.  -2. ]
 [ 0.5  0.   1.5  1.   0.  -0.5  1. ]
 [ 1.5  0.   3.5  0.   1.  -0.5  3. ]
 [ 0.5  1.  -0.5  0.   0.   0.5  1. ]]


In [132]:
tableau = sp.Matrix([[1., 2 + delta, -2, 0., 0., 0, 0.],
                    [0.5, 0., 1.5, 1., 0., -0.5, 1.],
                     [1.5, 0., 3.5, 0., 1., -0.5, 3.],
                     [0.5, 1., -0.5, 0., 0., 0.5, 1.]])

basic_var = [3, 4, 1]
display(tableau)
pivoting_symbolic(tableau, 3, 1)
tableau

Matrix([
[1.0, \delta + 2,   -2,   0,   0,    0,   0],
[0.5,          0,  1.5, 1.0,   0, -0.5, 1.0],
[1.5,          0,  3.5,   0, 1.0, -0.5, 3.0],
[0.5,        1.0, -0.5,   0,   0,  0.5, 1.0]])

Matrix([
[-0.5*\delta,   0, 0.5*\delta - 1.0,   0,   0, -0.5*\delta - 1.0, -1.0*\delta - 2.0],
[        0.5,   0,              1.5, 1.0,   0,              -0.5,               1.0],
[        1.5,   0,              3.5,   0, 1.0,              -0.5,               3.0],
[        0.5, 1.0,             -0.5,   0,   0,               0.5,               1.0]])